In [1]:
import numpy as np
import pandas as pd
import os, time, re
import pickle, gzip, datetime

from datetime import datetime

now = datetime.now()

import matplotlib.pyplot as plt
import seaborn as sns
color = sns.color_palette()
import matplotlib as mpl
%matplotlib inline


from sklearn import preprocessing as pp
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import log_loss
from sklearn.metrics import precision_recall_curve, average_precision_score
from sklearn.metrics import roc_curve, auc, roc_auc_score, mean_squared_error
import lightgbm as lgb


import tensorflow as tf
import keras

from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Activation, Dense, Dropout
from tensorflow.keras.layers import BatchNormalization, Input, Lambda
from tensorflow.keras.layers import Embedding, Flatten, dot
from tensorflow.keras import regularizers
from tensorflow.keras.losses import mse, binary_crossentropy

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

2025-09-26 18:10:14.829505: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1758910215.121442      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1758910215.202858      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


/kaggle/input/movie-ratings/ratings.csv


In [2]:
df=pd.read_csv("/kaggle/input/movie-ratings/ratings.csv")
df.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,1225734739
1,1,110,4.0,1225865086
2,1,158,4.0,1225733503
3,1,260,4.5,1225735204
4,1,356,5.0,1225735119


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33832162 entries, 0 to 33832161
Data columns (total 4 columns):
 #   Column     Dtype  
---  ------     -----  
 0   userId     int64  
 1   movieId    int64  
 2   rating     float64
 3   timestamp  int64  
dtypes: float64(1), int64(3)
memory usage: 1.0 GB


In [4]:
df.shape

(33832162, 4)

In [5]:
df.duplicated().sum()

0

In [6]:
df.isnull().sum()

userId       0
movieId      0
rating       0
timestamp    0
dtype: int64

In [7]:
#rating distribution across the dataset
# in real world setting it is obvious for the data to be skewed towards higher ratings

df1=df["rating"].value_counts()
df1=df1.reset_index()
df1["proportion"]=df1["count"]*100/len(df)
df1

,rating,count,proportion
0,4.0,8835955,26.117027
1,3.0,6400664,18.918874
2,5.0,4957902,14.654405
3,3.5,4465001,13.197504
4,4.5,3123055,9.231024
5,2.0,2146492,6.344531
6,2.5,1760733,5.204317
7,1.0,1013645,2.996099
8,0.5,566306,1.673869
9,1.5,562409,1.662350


In [30]:
#number of users
n_users_org=df["userId"].nunique()

In [31]:
#number of movies
n_movies_org=df["movieId"].nunique()

In collaborative filtering, you usually have a user–item interaction matrix:
Rows = users
Columns = items (movies/products)
Entries = rating (explicit) or interaction (implicit: click, view, purchase)

Most users don’t rate most items → so most entries are empty (missing).
That’s sparsity.

Formally:
sparsity = 1 − (number of observed ratings/number of possible ratings)

Our dataset is sparse almost entirely (>99%). 

Why sparsity matters
- High sparsity → harder to learn similarities (cold-start users/items).
- Too sparse → overfitting risk (not enough overlap).
- Dense subset (like a 1k×1k sample) → useful for prototyping.

To reduce this sparsity, will take a sample of movies/users with greater number of ratings/rated highest number of times(highest interactions), thereby overcoming:
1. Long tailed problem - where movies with 1 or 2 ratings will be ignored
2. Hence, avoiding cold start problem - will have no ratings to learn from
3. Hence, avoiding high sparsity

In [37]:
# Generate ratings matrix for train
ratings = np.zeros((n_users_org, n_movies_org))
for row in df.itertuples():
    ratings[row[1]-1, row[2]-1] = row[3]

IndexError: index 88124 is out of bounds for axis 1 with size 83239

In [ ]:
sparsity = float(len(ratings.nonzero()[0]))
sparsity /= (ratings.shape[0] * ratings.shape[1])
sparsity *= 100
print('Sparsity: {:4.2f}%'.format(sparsity))

In [10]:
#calculating sparsity
1-(len(df["rating"])/(df["userId"].nunique()*df["movieId"].nunique()))

0.9987719735903972

1. Movies are the "content backbone"
Some movies are rated thousands of times (popular blockbusters).
Others are rated only once or twice (obscure indie films).

If we include those rare movies, our matrix becomes very sparse and cold-start dominates.

=> That’s why we forcefully pick the top 1000 movies — we want only those with strong collaborative signal.

2. Users are more variable
We don’t want to always pick the same top 1000 most-active users.
Why? Because then your subset would be biased toward power users who rate hundreds or thousands of movies.

That subset wouldn’t reflect the variety of user behaviors (light raters vs. heavy raters).

=> So instead, you take a random sample of 1000 users (from the set of active users).

This balances the dataset.
=> Keeps diversity in user behaviors.
=> Makes sure you don’t just overfit to super-raters.

3. The balance

So:
Movies → fix the set to the most rated ones (dense overlap).
Users → sample a representative group (avoid bias, retain variety).

This creates a dense yet balanced subset.

4. What if we didn’t do this?
If we randomly sample movies → we’ll likely include many with just 1–2 ratings → kills overlap, matrix too sparse.
If we take only the very top users → our dataset only reflects “power raters”, and recommendations won’t generalize well.

So this asymmetric choice is a practical hack:
Items: deterministic top-k → ensures enough data per item => to guarantee density (avoid long-tail)
Users: random sample from active pool → ensures variety =>to preserve diversity and prevent bias toward only power-users.

In [11]:
#top 1000 highly rated movies (highest number of times)
movieIndex=df.groupby("movieId").count().sort_values(by="rating",ascending=False)[0:1000].index
movieIndex

ratingDF=df[df['movieId'].isin(movieIndex)]
ratingDF.count()

userId       20152541
movieId      20152541
rating       20152541
timestamp    20152541
dtype: int64

In [12]:
#top 1000 highest interactions
userIndex=ratingDF.groupby("userId").count().sort_values(by="rating",ascending=False).sample(n=1000, random_state=2018).index
userIndex

ratingDF1=ratingDF[ratingDF['userId'].isin(userIndex)]
ratingDF1.count()

userId       66616
movieId      66616
rating       66616
timestamp    66616
dtype: int64

In [13]:
#reindexing from 1 to 1000
movies=ratingDF1.movieId.unique()
moviesDF=pd.DataFrame(movies, columns=["originalMovieId"])
moviesDF["newMovieId"]=moviesDF.index+1
moviesDF

users=ratingDF1.userId.unique()
usersDF=pd.DataFrame(users,columns=["originalUserId"])
usersDF["newUserId"]=usersDF.index+1
usersDF

,originalUserId,newUserId
0,4,1
1,136,2
2,583,3
3,1193,4
4,1240,5
...,...,...
995,327543,996
996,329204,997
997,329479,998
998,329604,999


In [14]:
#merging dataframes
ratingDF1=ratingDF1.merge(moviesDF, left_on="movieId", right_on="originalMovieId")
ratingDF1

,userId,movieId,rating,timestamp,originalMovieId,newMovieId
0,4,260,5.0,1442455519,260,1
1,4,318,5.0,1442455559,318,2
2,4,356,5.0,1442455449,356,3
3,4,595,4.5,1442456131,595,4
4,4,858,3.0,1442455657,858,5
...,...,...,...,...,...,...
66611,330693,708,3.0,843924587,708,224
66612,330693,778,5.0,843925021,778,227
66613,330693,838,3.0,843925062,838,232
66614,330693,1079,5.0,843925041,1079,401


In [15]:
ratingDF1.drop("originalMovieId", inplace=True, axis=1)

In [16]:
ratingDF1=ratingDF1.merge(usersDF, left_on="userId", right_on="originalUserId")
ratingDF1.head()

,userId,movieId,rating,timestamp,newMovieId,originalUserId,newUserId
0,4,260,5.0,1442455519,1,4,1
1,4,318,5.0,1442455559,2,4,1
2,4,356,5.0,1442455449,3,4,1
3,4,595,4.5,1442456131,4,4,1
4,4,858,3.0,1442455657,5,4,1


In [17]:
ratingDF1.drop("originalUserId", inplace=True, axis=1)

In [18]:
ratingDF1

,userId,movieId,rating,timestamp,newMovieId,newUserId
0,4,260,5.0,1442455519,1,1
1,4,318,5.0,1442455559,2,1
2,4,356,5.0,1442455449,3,1
3,4,595,4.5,1442456131,4,1
4,4,858,3.0,1442455657,5,1
...,...,...,...,...,...,...
66611,330693,708,3.0,843924587,224,1000
66612,330693,778,5.0,843925021,227,1000
66613,330693,838,3.0,843925062,232,1000
66614,330693,1079,5.0,843925041,401,1000


In [23]:
1-(len(ratingDF1["rating"])/(ratingDF1["userId"].nunique()*ratingDF1["movieId"].nunique()))

0.933384

In [27]:
n_users=ratingDF1["userId"].nunique() 
n_movies=ratingDF1["movieId"].nunique() 
n_ratings=len(ratingDF1)

n_users, n_movies, n_ratings

(1000, 1000, 66616)

In [22]:
#generating train, test and val sets
train_set, test_set=train_test_split(ratingDF1, test_size=0.2, shuffle=True, random_state=42)
train_set, val_set=train_test_split(train_set, test_size=0.2, shuffle=True, random_state=42)
train_set.shape, val_set.shape, test_set.shape

((42633, 6), (10659, 6), (13324, 6))

In [28]:
# Generate ratings matrix for train
ratings_train = np.zeros((n_users, n_movies))
for row in train_set.itertuples():
    ratings_train[row[6]-1, row[5]-1] = row[3]

In [29]:
sparsity = float(len(ratings_train .nonzero()[0]))
sparsity /= (ratings_train.shape[0] * ratings_train.shape[1])
sparsity *= 100
print('Sparsity: {:4.2f}%'.format(sparsity))

Sparsity: 4.26%
